In [2]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import random
import pandas as pd

In [6]:
class individuo:
  def __init__(self,k_max,k_min,x_min,x_max,d):
    self.cromosoma = list(np.random.random(k_max))
    self.checar(k_max,k_min)

    for j in range(k_max): #creamos los centroides
      self.cromosoma.append(list(np.random.uniform(low=x_min,high=x_max,size=d)))
  def checar(ind,k_max,k_min):
    ind.activos = [i for i in range(k_max) if ind.cromosoma[i] > 0.5]
    while len(ind.activos) < k_min:
      ind.cromosoma = list(np.random.random(k_max))
      ind.activos = [i for i in range(k_max) if ind.cromosoma[i] > 0.5]
  cs = None
  particion = None

In [7]:
class individuo_vacio:
  cromosoma = []
  activos = []
  cs = None
  particion = None

In [8]:
def distancia(x1 , x2, flag = False ):
  '''la bandera nos indica si se calcula distancia entre datos 
  o entre un dato y un centroide, cuando es True indica distancia
  entre 2 puntos'''
  global distanciasP
  if flag == False:
    return( dot(x1, x2)/(norm(x1)*norm(x2)) )
  else:
    try:
      return(distanciasP[str(x1+x2)])
    except:
      try:
        return(distanciasP[str(x2+x1)])
      except:
        dist = dot(x1, x2)/(norm(x1)*norm(x2))
        distanciasP[str(x2+x1)] = dist
        distanciasP[str(x1+x2)] = dist
        return(distanciasP[str(x1+x2)])

In [8]:
distanciasP={}
distancia([4,5],[1,2],True)

0.9778024140774094

In [9]:
def obtener_elementos(particion,cluster,datos):
  '''
  Devuelve los elemetos que forman parte del cluster especificado,
  ya que los guardamos en una lista simple, donde el indice indica 
  el numero del elemento y el valor indica el cluster al que pertenece.
  '''
  return [datos[i] for i in range(len(particion)) if particion[i] == cluster]

In [10]:
def repartir_datos(len_datos,activos):
  r = []
  c = 0
  while len(r) < len_datos:
    r.append(activos[c])
    if c !=len(activos)-1:
      c+=1
    else:
      c=0
  return(r)

In [11]:
#version funcional
def completar_individuo(individuo , datos : list,k_max : int, eps = 0.0001):
  '''esta funcion agrega los datos que le hacen falta a los individuos cuando
  son recien creados, le agrega su particion de elementos y el valor de que
  toma en la funcion objetivo '''

  'primero agregamos la particion'
  particion = []
  for dato in datos:
    distancias = [ distancia( dato,individuo.cromosoma[k_max+i] ) if individuo.cromosoma[i] > 0.5 else -1.1 for i in range(k_max)  ]
    #print('distancias',distancias)
    particion.append(distancias.index(max(distancias))) # la similitud da mayor puntaje a objetos mas cercanos, por eso usamos max
  individuo.particion = particion

  'Algunos cromosomas dan errores, aqui verificamos si es el caso, si asi fuera, lo solucionamos cambiando la particion'
  for grupo in individuo.activos:
    cant_ele = sum([1 for i in individuo.particion if i == grupo])
    'un cromosoma defectuosos es el que contiene algun centro con menos de 2 elementos, verificamos si esto pasa'
    if cant_ele < 2:
      individuo.particion = repartir_datos(len(datos),individuo.activos)
      break
  #print('particion ind',individuo.particion)
  individuo.cs = 1 / (cs(individuo,datos,k_max) + eps)

In [12]:
def calcular_centroide(centro,elementos):
  return(list(np.mean(elementos,axis = 0)))

In [21]:
def calcular_dist(datos):
  global distanciasP
  c=0
  for i in range(len(datos)):
    for j in range(i+1,len(datos)):
      c+=1
      x1 = datos[i]
      x2 = datos[j]
      dist = dot(x1, x2)/(norm(x1)*norm(x2))
      distanciasP[str(x2+x1)] = dist
      distanciasP[str(x1+x2)] = dist
  print('deberian de haberse calculado todas las distancias entre puntos')
  print('contador',c)


In [19]:
def ACDE(cr_max, cr_min, k_max, k_min, pop_size, x_min, x_max, d : int, max_eval, datos : list,eps : float = 0.001):
  '''
  cr_max es el valor maximo que puede tomar la constante de cruza
  cr_min es el minimo valor que puede tomar la CR
  k_max el numero maximo de grupos que puede haber
  k_min el minimo de rupos que deben de considerarse
  pop_size tamaño de la poblacion
  x_max y x_min sirven para delimitar el rango de valores que pueden
  tener los datos, por ejemplo, estan en el rango (0,2)
  d es la dimension de los datos R2 : (x,y), R3 : (x,y,z) etc
  max_eval es la cantidad maxima de llamadas a la funcion aptitud CS
  datos es una lista con los datos a ordenar
  ADVERTENCIA LA CANTIDAD DE DATOS DEBE SER AL MENOS 2 VECES LA CANTIDAD DE GRUPOS
  '''
  global distanciasP
  distanciasP= {}
  calcular_dist(datos)
  print('el diccionario tiene', len(distanciasP))

  'creamos la poblacion inicial y le agregamos sus particiones y valores en CS'
  poblacion = [individuo(k_max,k_min,x_min,x_max,d) for i in range(pop_size)]
  for ind in poblacion:
    completar_individuo(ind, datos, k_max,eps)
  'iteramos el algoritmo t generaciones, para saber cuantas deben ser, al numero de evaluaciones restamos pop_size y dividimos entre pop_size'
  iter = (max_eval*pop_size)//pop_size
  best = sorted(poblacion, key= lambda x: x.cs, reverse=True)[0]

  'la siguiente lista es solo para darse una idea de cuanto va avanzando el algoritmo'
  avance = np.linspace(10,1000,num=100)


  for t in range(1,iter+1):
    cr = (cr_max-cr_min) * ((iter-t) / iter)
    F = 0.5*(1+np.random.random())
    'recorremos individuo a individuo para aplicar ED'
    for ind in poblacion:
      intento = individuo_vacio()
      a,b,c = random.sample(poblacion,3)
      'hacemos la cruza y mutacion para darle valores al individuo intento'
      l = []
      for i in range(len(ind.cromosoma)):
        if np.random.random() < cr:
          if i < k_max:
            l.append(c.cromosoma[i] + F*(a.cromosoma[i]-b.cromosoma[i]) )
          else:
            l.append( [c.cromosoma[i][j] + F*(a.cromosoma[i][j]-b.cromosoma[i][j]) for j in range(d)     ]   )
        else:
          l.append(ind.cromosoma[i])
      intento.cromosoma = l
      intento.activos = [j for j in range(k_max) if intento.cromosoma[j] > 0.5]
      'existe la posibilidad, de que al cruzar no se cumpla que hayan almenos k_min cluster activos, si esto pasa reiniciamos los coeficientes'
      while len(intento.activos) < k_min:
        intento.cromosoma[:k_max] = np.random.random(k_max)
        intento.activos = [j for j in range(k_max) if intento.cromosoma[j] > 0.5]
      completar_individuo(intento,datos,k_max,eps)
      if intento.cs > ind.cs:
        poblacion[poblacion.index(ind)]= intento
      if intento.cs > best.cs:
        best = intento
    if t in avance:
      print('vamos en la iteracion',t,'avance: ',(t*10)/1000,'%')
    #print(t)
  print('mejor',best.cromosoma,'puntiacion',best.cs)

In [ ]:
datos = [[4,5],[1,3],[0,2],[1,1],[5,4],[15,3],[1,7],[2,5] ]

In [ ]:
ACDE(cr_max=1,cr_min=0.5,k_max=3,k_min=2,pop_size=5,x_min=2,x_max=5,d = 2,max_eval=20,datos=datos,eps=0.001)

mejor [0.9032407491002791, 0.6973664637344199, 0.7182817043527832, [2.0, 4.333333333333333], [2.6666666666666665, 4.0], [7.5, 2.5]] puntiacion 1.4747385050162212


In [15]:
#version final funcional
def cs(individuo, datos : list,k_max):
  dividendo = []
  divisor = []
  centros = individuo.cromosoma[k_max::]
  for centro in individuo.activos:
    elementos = obtener_elementos(individuo.particion,centro,datos)
    centros[centro] = calcular_centroide(centro,elementos)
    'vamos a calcular la distancia mas grande de un punto a cualquier otro, para todos los puntos en el grupo'
    distancias_puntos = []
    for i in range(len( elementos)):
      distancias_puntos.append( min([distancia(elementos[i],elementos[x2],True) for x2 in range(len(elementos)) if x2 != i ] )    )
    dividendo.append(np.mean(distancias_puntos))
    'ahora calculamos las mimimas distancias entre los centroides de cada grupo'
    divisor.append( max([distancia(centros[centro],centros[x1]) for x1 in individuo.activos if x1 != centro] )  )
  'actualizamos la informacion del individuo, ya que alteramos una copia de sus datos'
  individuo.cromosoma[k_max::] = centros
  return(np.mean(dividendo) / np.mean(divisor))

In [3]:
datos = pd.read_csv('/content/drive/MyDrive/csv/iris_data.csv',header=None);
datos.columns = [0,1,2,3,'clase']
datos.drop(['clase'],axis = 1,inplace=True)
datos = datos.to_numpy()

In [17]:
datos[0:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [31]:
print('maximo',max([x for y in datos for x in y])  )
print('minimo',min([x for y in datos for x in y])  )

maximo 7.9
minimo 0.1


In [17]:
len(datos)

150

In [ ]:
ACDE(cr_max=1,cr_min=0.5,k_max=20,k_min=2,pop_size=10,x_min=0.1,x_max=7.9,d = 4,max_eval=100,datos=datos,eps=0.001)

In [24]:
#version beta
def completar_individuo(individuo , datos : list,k_max : int):
  '''esta funcion agrega los datos que le hacen falta a los individuos cuando
  son recien creados, le agrega su particion de elementos y el valor de que
  toma en la funcion objetivo '''

  'primero agregamos la particion'
  particion = []
  for dato in datos:
    distancias = [ distancia( dato,individuo.cromosoma[k_max+i] ) if individuo.cromosoma[i] > 0.5 else -1.1 for i in range(k_max)  ]
    particion.append(distancias.index(max(distancias))) # la similitud da mayor puntaje a objetos mas cercanos, por eso usamos max
  individuo.particion = particion

  'Algunos cromosomas dan errores, aqui verificamos si es el caso, si asi fuera, lo solucionamos cambiando la particion'
  for grupo in individuo.activos:
    cant_ele = sum([1 for i in individuo.particion if i == grupo])
    #elementos = obtener_elementos(individuo.particion,grupo,datos)
    'un cromosoma defectuosos es el que contiene algun centro con menos de 2 elementos, verificamos si esto pasa'
    if cant_ele < 2:
      individuo.activos.remove(grupo)

  individuo.cs = cs(individuo,datos,k_max)


In [ ]:
#version beta
def cs(individuo, datos : list,k_max):
  dividendo = []
  divisor = []
  centros = individuo.cromosoma[k_max::]
  for centro in individuo.activos:
    elementos = obtener_elementos(individuo.particion,centro,datos)
    centros[centro] = calcular_centroide(centro,elementos)
    'vamos a calcular la distancia mas grande de un punto a cualquier otro, para todos los puntos en el grupo'
    distancias_puntos = []
    for punto in elementos:
      distancias_puntos.append( min([distancia(punto,x2) for x2 in elementos if x2 != punto ] )    )
    dividendo.append(np.mean(distancias_puntos))
    'ahora calculamos las mimimas distancias entre los centroides de cada grupo'
    divisor.append( max([distancia(centros[centro],centros[x1]) for x1 in individuo.activos if x1 != centro] )  )
  'actualizamos la informacion del individuo, ya que alteramos una copia de sus datos'
  individuo.cromosoma[k_max::] = centros
  return(np.mean(dividendo) / np.mean(divisor))